In [4]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

# Veri yolu - Klasördeki verilerin kategori alt klasörlerde bulunduğunu varsayalım.
data_dir = 'Soil types/'

# Veri dönüşümleri (örnek olarak)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Görselleri istediğiniz boyuta dönüştürebilirsiniz
    transforms.ToTensor(),          # Görselleri tensöre dönüştür
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizasyon (örnek)
])

# Veri yükleyici parametreleri
batch_size = 32
num_workers = 4

# Veriyi yükleyin
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [5]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Giriş boyutunu uygun şekilde ayarlayın
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Modeli oluştur
num_classes = len(dataset.classes)  # Sınıf sayısı otomatik olarak belirlenir
model = CNNModel(num_classes)


In [8]:
# Eğitim parametreleri
learning_rate = 0.001
num_epochs = 10

# Kayıp fonksiyonu ve optimizasyon fonksiyonu
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
correct = 0
total = 0

# Eğitim döngüsü
for epoch in range(num_epochs):
    for images, labels in dataloader:
        # İleri geçiş
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Geriye doğru yayılım ve parametre güncelleme
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()} Accuracy: {accuracy}')


Epoch [1/10] Loss: 0.3367578983306885 Accuracy: 84.66453674121406
Epoch [2/10] Loss: 0.1407817304134369 Accuracy: 90.57507987220447
Epoch [3/10] Loss: 0.04745684936642647 Accuracy: 93.07774227902023
Epoch [4/10] Loss: 0.00037880995660088956 Accuracy: 94.72843450479233
Epoch [5/10] Loss: 0.022711701691150665 Accuracy: 95.71884984025559
Epoch [6/10] Loss: 0.0037627951242029667 Accuracy: 96.3258785942492
Epoch [7/10] Loss: 0.0026297406293451786 Accuracy: 96.85075308078503
Epoch [8/10] Loss: 0.0002065466542262584 Accuracy: 97.2444089456869
Epoch [9/10] Loss: 0.00020160162239335477 Accuracy: 97.55058572949947
Epoch [10/10] Loss: 0.00013027996465098113 Accuracy: 97.79552715654953


In [9]:
# Modeli kaydet
torch.save(model.state_dict(), 'soil.pth')

In [10]:
# Model yapısını oluştur
model = CNNModel(num_classes)  # Model yapısını aynı şekilde oluşturmalısınız

# Kaydedilmiş parametreleri yükle
model.load_state_dict(torch.load('soil.pth'))

# Modeli tahminlerde kullanabilirsiniz
model.eval()  # Modeli tahmin modunda etkinleştirin
# Burada tahminlerinizi yapabilirsiniz


CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=5, bias=True)
)

In [16]:
import cv2
from PIL import Image

In [17]:
image_path = 'blacksoil.jpg'  # Test görüntüsünün dosya yolu
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Görüntüyü BGR'den RGB'ye dönüştür
image_pil = Image.fromarray(image)

In [18]:
# Giriş görüntüsünü modele uygun hale getir
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_image = transform(image_pil)
input_image = input_image.unsqueeze(0)  # Batch boyutunu ekleyin (1 görüntü için)

In [19]:
with torch.no_grad():
    output = model(input_image)

# Sınıflandırma sonuçlarını al
_, predicted = torch.max(output, 1)

# Sınıf tahminini bul
class_index = predicted.item()
class_label = dataset.classes[class_index]  # 'dataset' veri kümesinin sınıflarını içerdiğini varsayalım

print(f'Tahmin edilen sınıf: {class_label}')

Tahmin edilen sınıf: Black Soil
